In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
workspace = '/content/gdrive/MyDrive/ONNXAnalyser/onnx_models'

Mounted at /content/gdrive


In [3]:
!pip install onnx numpy
!pip install onnxruntime
!pip install plotly==4.14.3
!pip install kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 77.3 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.13.1
    Uninstalling plotly-5.13.1:
      Successfully uninstalled plotly-5.13.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.2 MB/s eta 0:00:00


In [ ]:
import onnx
import onnx.shape_inference
import numpy as np
from collections import defaultdict

# Dictionary to store initializer tensors and their shapes
initializer_dict = {}

def print_model_compute(model_path):
    model = onnx.load(model_path)
    # Perform shape inference on the model
    inferred_model = onnx.shape_inference.infer_shapes(model)

    graph = inferred_model.graph

    for initializer in model.graph.initializer:
        initializer_dict[initializer.name] = tuple(d for d in initializer.dims)

    # print("Input Parameters:")
    # for input_param in graph.input:
    #     name = input_param.name
    #     shape = get_shape(graph, name)
    #     dtype = input_param.type.tensor_type.elem_type
    #     print(f"Name: {name}, Shape: {shape}, Data Type: {dtype}")

    total_flops = 0
    model_summary = defaultdict(int)
    print("\nLayer Dimensions:")
    for i, node in enumerate(graph.node):
        op_type = node.op_type
        output_name = node.output[0]
        model_summary[op_type] += 1
        if op_type == 'Conv':
            input_name = node.input[0]
            input_weight = node.input[1]
            input_shape = get_shape(graph, input_name)
            weight_shape = get_shape(graph, input_weight)
            output_shape = get_shape(graph, output_name)
            # print(node.attribute)
            # kernel_shape = node.attribute[2].ints
            for attribute in node.attribute:
              if attribute.name == 'kernel_shape':
                kernel_shape = attribute.ints
              if attribute.name == 'strides':
                stride_shape = attribute.ints
            # stride_shape = [attribute.ints for attribute in node.attribute if attribute.name == 'strides']
            # print(stride_shape)
            # stride_shape = node.attribute[4].ints
            # input dimensions
            C_in = input_shape[1]
            # output dimensions
            C_out = output_shape[1]
            H_out = output_shape[2]
            W_out = output_shape[3]
            # kernel dimensions
            H_k = kernel_shape[0]
            W_k = kernel_shape[1]
            # stride
            stride = stride_shape[0]
            # compute flops
            flops = C_in * H_out * W_out  * C_out * H_k * W_k / (stride * stride)
            total_flops += flops
            print(f"{i} Conv Layer: Input Shape: {input_shape}, weight: {weight_shape}, Output Shape: {output_shape}, flops: {flops}")

        elif op_type == 'Gemm':
            input_name = node.input[0]
            input_weight = node.input[1]
            input_shape = get_shape(graph, input_name)
            weight_shape = get_shape(graph, input_weight)
            output_shape = get_shape(graph, output_name)
            N_in = input_shape[1]
            H_w, W_w = weight_shape[0], weight_shape[1]
            flops = N_in * H_w * W_w
            total_flops += flops
            print(f"{i} Gemm {input_name} Layer: Input Shape: {input_shape}, weight: {weight_shape}, Output Shape: {output_shape}, flops: {flops}")

        elif op_type == 'AveragePool':
            input_name = node.input[0]
            input_shape = get_shape(graph, input_name)
            output_shape = get_shape(graph, output_name)
            kernel_shape = node.attribute[1].ints
            C_in = input_shape[1]
            H_out = output_shape[2]
            W_out = output_shape[3]
            H_k = kernel_shape[0]
            W_k = kernel_shape[1]
            flops = C_in * H_out * W_out * H_k * W_k
            total_flops += flops
            print(f"{i} AveragePool {input_name} Layer: Input Shape: {input_shape}, Output Shape: {output_shape}, flops: {flops}")

        elif op_type == 'GlobalAveragePool':
            input_name = node.input[0]
            input_shape = get_shape(graph, input_name)
            output_shape = get_shape(graph, output_name)
            C_in = input_shape[1]
            H_in = input_shape[2]
            W_in = input_shape[3]
            H_out = output_shape[2]
            W_out = output_shape[3]
            flops = C_in * H_out * W_out * H_in * W_in
            total_flops += flops
            print(f"{i} GlobalAveragePool {input_name} Layer: Input Shape: {input_shape}, Output Shape: {output_shape}, flops: {flops}")

        elif op_type == 'Add':
            input1_name = node.input[0]
            input2_name = node.input[1]
            input1_shape = get_shape(graph, input1_name)
            input2_shape = get_shape(graph, input2_name)
            output_shape = get_shape(graph, output_name)
            C = input_shape[1]
            H = input_shape[2]
            W = input_shape[3]
            flops = C * H * W
            total_flops += flops
            print(f"{i} Add Layer: Input1 Shape: {input1_shape}, Input2 Shape: {input2_shape}, Output Shape: {output_shape}, flops: {flops}")

        elif op_type == 'MatMul':
            input1_name = node.input[0]
            input2_name = node.input[1]
            input1_shape = get_shape(graph, input1_name)
            input2_shape = get_shape(graph, input2_name)
            output_shape = get_shape(graph, output_name)
            H_in, W_in = input_shape[0], input_shape[1]
            H_out, W_out = output_shape[0], output_shape[1]
            flops = H_in * W_in * H_out * W_out
            total_flops += flops
            print(f"{i} MatMul Layer: Input1 Shape: {input1_shape}, Input2 Shape: {input2_shape}, Output Shape: {output_shape}, flops: {flops}")
    # print model summary
    total_operations = 0
    print("model summary: {}".format(model_path))
    for op_type, op_num in model_summary.items():
      print("{}: {}".format(op_type, op_num))
      total_operations += op_num
    print("total operations: {}".format(total_operations))
    return total_flops

def get_shape(graph, name):
    for input_param in graph.input:
        if input_param.name == name:
            shape = [dim.dim_value for dim in input_param.type.tensor_type.shape.dim]
            # print('input_param:', input_param)
            return shape
    for value_info in graph.value_info:
        if value_info.name == name:
            shape = [dim.dim_value for dim in value_info.type.tensor_type.shape.dim]
            # print('value_info:', value_info)
            if len(shape) != 0:
              return shape
    for output_param in graph.output:
        if output_param.name == name:
            shape = [dim.dim_value for dim in output_param.type.tensor_type.shape.dim]
            # print('output_param:', output_param)
            return shape

    if name in initializer_dict:
        weight = initializer_dict[name]
        # print('initializer_dict:', weight)
        return weight

    return None

# Path to the ONNX model file
model_path = workspace + '/mobilenetv2-7.onnx'

# Print input parameters and layer dimensions
print_model_compute(model_path)



Layer Dimensions:
0 Conv Layer: Input Shape: [1, 3, 224, 224], weight: [32, 3, 3, 3], Output Shape: [1, 32, 112, 112], flops: 2709504.0
3 Conv Layer: Input Shape: [1, 32, 112, 112], weight: [32, 32, 1, 1], Output Shape: [1, 32, 112, 112], flops: 12845056.0
6 Conv Layer: Input Shape: [1, 32, 112, 112], weight: [32, 1, 3, 3], Output Shape: [1, 32, 112, 112], flops: 115605504.0
9 Conv Layer: Input Shape: [1, 32, 112, 112], weight: [16, 32, 1, 1], Output Shape: [1, 16, 112, 112], flops: 6422528.0
11 Conv Layer: Input Shape: [1, 16, 112, 112], weight: [96, 16, 1, 1], Output Shape: [1, 96, 112, 112], flops: 19267584.0
14 Conv Layer: Input Shape: [1, 96, 112, 112], weight: [96, 1, 3, 3], Output Shape: [1, 96, 56, 56], flops: 65028096.0
17 Conv Layer: Input Shape: [1, 96, 56, 56], weight: [24, 96, 1, 1], Output Shape: [1, 24, 56, 56], flops: 7225344.0
19 Conv Layer: Input Shape: [1, 24, 56, 56], weight: [144, 24, 1, 1], Output Shape: [1, 144, 56, 56], flops: 10838016.0
22 Conv Layer: Input Sh

7590732416.0

In [ ]:
import os
onnx_models = os.listdir(workspace)

compute_estimate = {}

onnx_models = ['bvlcalexnet-12.onnx','vgg16-12.onnx','vgg19-7.onnx', 'inception-v1-12.onnx', 'inception-v2-9.onnx',
               'googlenet-12.onnx','resnet18-v1-7.onnx', 'resnet34-v1-7.onnx', 'resnet50-v1-7.onnx', 'resnet101-v1-7.onnx', 'resnet152-v1-7.onnx',
               'densenet-9.onnx', 'squeezenet1.0-12.onnx', 'mobilenetv2-7.onnx', 'shufflenet-v2-10.onnx', 'efficientnet-lite4-11.onnx']

# error: 'inception-v1-3.onnx',

for onnx_model in onnx_models:
  # Path to the ONNX model file
  if 'onnx' not in onnx_model: # or 'vgg' in onnx_model:
    continue
  print('Model:', onnx_model)
  model_path = workspace + '/' + onnx_model
  # Input data as a NumPy array
  input_data = np.random.randn(1, 3, 224, 224).astype(np.float32)  # Adjust the shape and data type as per your model's input requirements
  # Print intermediate results
  compute_estimate[onnx_model[:-5]] = print_model_compute(model_path)



Model: bvlcalexnet-12.onnx

Layer Dimensions:
0 Conv Layer: Input Shape: [1, 3, 224, 224], weight: (96, 3, 11, 11), Output Shape: [1, 96, 54, 54], flops: 6351048.0
4 Conv Layer: Input Shape: [1, 96, 26, 26], weight: (256, 48, 5, 5), Output Shape: [1, 256, 26, 26], flops: 415334400.0
8 Conv Layer: Input Shape: [1, 256, 12, 12], weight: (384, 256, 3, 3), Output Shape: [1, 384, 12, 12], flops: 127401984.0
10 Conv Layer: Input Shape: [1, 384, 12, 12], weight: (384, 192, 3, 3), Output Shape: [1, 384, 12, 12], flops: 191102976.0
12 Conv Layer: Input Shape: [1, 384, 12, 12], weight: (256, 192, 3, 3), Output Shape: [1, 256, 12, 12], flops: 127401984.0
16 Gemm OC2_DUMMY_0 Layer: Input Shape: [1, 9216], weight: (4096, 9216), Output Shape: [1, 4096], flops: 347892350976
19 Gemm fc6_3 Layer: Input Shape: [1, 4096], weight: (4096, 4096), Output Shape: [1, 4096], flops: 68719476736
22 Gemm fc7_3 Layer: Input Shape: [1, 4096], weight: (1000, 4096), Output Shape: [1, 1000], flops: 16777216000
model su

In [ ]:
import pickle as pkl
file_path = workspace + '/compute_estimates.pkl'  # Replace with the desired file path

# Open the file in binary mode for writing
with open(file_path, 'wb') as file:
    pkl.dump(compute_estimate, file)

In [ ]:
import pickle as pkl
file_path = workspace + '/compute_estimates.pkl'  # Replace with the desired file path

# Open the file in binary mode for writing
with open(file_path, 'rb') as file:
    compute_estimate = pkl.load(file)

In [ ]:
import plotly.graph_objects as go
from plotly.offline import plot
# Extract keys and values from the dictionary
categories = list(compute_estimate.keys())
counts = list(compute_estimate.values())

# Create a bar trace for the histogram
trace = go.Bar(x=categories, y=counts)

# Create the layout for the histogram
layout = go.Layout(title={'text': 'Network Architecture vs FLOPs',
                          'x': 0.5,  # Align title to the center horizontally
                   },
                   xaxis=dict(title='Network Architecture'),
                   yaxis=dict(title='FLOPs',
                   type='log'))

# Create the figure and add the trace
fig = go.Figure(data=[trace], layout=layout)

# Show the figure
fig.show()
fig.write_image('compute.jpg')
fig.write_html('image.html')

In [ ]:
!pip install kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pickle as pkl
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

compute_path = workspace + '/compute_estimates.pkl'  # Replace with the desired file path
memory_path = workspace + '/memory_estimates.pkl'  # Replace with the desired file path

# Load the compute data from pickle file
with open(compute_path, 'rb') as file:
    compute_data = pkl.load(file)

# Load the memory data from pickle file
with open(memory_path, 'rb') as file:
    memory_data = pkl.load(file)

# Extract x and y values from dictionaries
x = list(compute_data.values())
y = list(memory_data.values())
labels = list(compute_data.keys())  # Use keys as labels

# Create the graph
fig = go.Figure()

# Plotting the scatter plot
for i in range(len(x)):
    fig.add_trace(go.Scatter(x=[x[i]], y=[y[i]], mode='markers', name=labels[i],marker=dict(size=15, symbol=i)))

# # Plotting the scatter plot
# fig.add_trace(go.Scatter(x=x, y=y, mode='markers', text=labels))

# Customize the layout
fig.update_layout(
    title={
        'text': 'Compute (FLOPs) vs Memory Bandwidth',
        'x': 0.5,  # Align title to the center horizontally
    },
    xaxis_title='Compute (FLOPs)',
    yaxis_title='Memory Bandwidth/Inference',
    xaxis_type='log',  # Set x-axis to log scale
    showlegend=True,  # Display legend
)

# Display the graph
fig.show()
fig.write_image('compute_vs_bandwidth.jpg')
fig.write_html('image.html')

In [28]:
import pickle as pkl
import plotly.graph_objects as go

compute_path = workspace + '/compute_estimates.pkl'  # Replace with the desired file path
memory_path = workspace + '/memory_estimates.pkl'  # Replace with the desired file path

# Load the compute data from the pickle file
with open(compute_path, 'rb') as file:
    compute_data = pkl.load(file)

# Load the memory data from the pickle file
with open(memory_path, 'rb') as file:
    memory_data = pkl.load(file)

# Extract x and y values from dictionaries
x = list(compute_data.values())
y = list(memory_data.values())
labels = list(compute_data.keys())  # Use keys as labels

# Create the graph
fig = go.Figure()

# Plotting the scatter plot with adjusted text position
for i in range(len(x)):
    fig.add_trace(go.Scatter(
        x=[x[i]],
        y=[y[i]],
        mode='markers+text',
        name=labels[i],
        text=[labels[i]],
        textposition='bottom right',  # Adjust text position relative to markers
        marker=dict(size=15, symbol=i)
    ))

# Customize the layout
fig.update_layout(
    title={
        'text': 'Compute (FLOPs) vs Memory Bandwidth',
        'x': 0.5,  # Align title to the center horizontally
    },
    xaxis_title='Compute (FLOPs)',
    yaxis_title='Memory Bandwidth/Inference',
    xaxis_type='log',  # Set x-axis to log scale
    showlegend=True,  # Display legend
)

# Display the graph
fig.show()
# Set larger image dimensions and write the image
image_width = 4800
image_height = 1200
fig.write_image('compute_vs_bandwidth.jpg')
fig.write_html('image.html')
